# Obtenção dos dados

#### Separamos esse trecho do código para deixar mais organizado, além de não precisar fazer a extração dos dados diretamente do MYSQL em cada execução do código principal.

Escolhemos trabalhar com a base de dados criada para a A1, que contém dados nutricionais de lanches de três redes de fast food distintas: Mc Donalds, Burguer King e Starbucks. O primeiro passo, então, foi carregar os dados que estavam no MySQL para o Python utilizando as bibliotecas Pandas e Pymysql.

In [1]:
import pandas as pd
import pymysql
import getpass

Criação da conexão com o banco local:

In [2]:
p = getpass.getpass()
conexao = pymysql.connect(
    host='localhost',
    user='root',
    port=3306,
    passwd=p,
    db="fast_food")

········


Queremos fazer uma visualização que permita comparar os valores nutricionais dos lanches (através do tamanho das arestas), para isso importamos todos os lanches, sua tabela nutricional, a rede a que pertence e sua categoria. Esses últimos serão úteis para filtrar os dados.

In [3]:
query = """
SELECT p.name, n.*, c.name as "categoria", p.chain_id_chain as "rede"
FROM nutricional_units n
INNER JOIN product p ON n.id_nutricional_units = p.nutricional_units_id_nutricional_units
INNER JOIN category c ON p.category_id_category = c.id_category;
"""
df = pd.read_sql(query, con=conexao, index_col = "name")

# Tratamento dos dados

#### Para que a visualização do grafo ficasse da maneira que planejamos, precisamos tratar os dados obtidos.

Após vários testes usando nossa base completa, percebemos que, para o objetivo determinado anteriormente, fazia mais sentido filtrar melhor a base de dados. As visualizações do grafo, além do processamento do código no geral, era prejudicado pelo volume orginal. Além disso, as três redes de restaurante presentes na base, o "Starbucks" era a que fazia menos sentido ser comparada com as demais ("MC Donalds" e "Burguer King"). Por isso, optamos por remover os dados dessa rede do dataset. Assim, as colunas "sal" e "cafeina" também foram deletadas, pois os únicos valores não-nulos destas eram de dados do "Starbucks".

In [4]:
df = df.drop(['id_nutricional_units','salt','caffeine'], axis=1)
df = df[df["rede"]!=3]

Queremos que o tamanho (grossura) da aresta represente a quantidade presente do nutriente no lanche conectado. Como o range dentre os diferentes nutrientes é muito diferente, precisamos normalizar os dados para melhor visualização:

In [5]:
col=[x for x in df.columns[0:len(df.columns)-2]] #pega os nomes das colunas de nutrientes (-2 exclui "categoria" e "rede")
nutrientes = df[col].copy() # criar uma cópia, somente com as colunas dos nutrientes

nutri_norm = (nutrientes - nutrientes.mean()) / nutrientes.std() #normaliza os nutrientes
nutri_min = abs(nutri_norm.min())

nutri_norm = nutri_norm + nutri_min + 1 #deixar todos os valores > 0

for nutriente in col:
    df[nutriente] = nutri_norm[nutriente] # aplica a normalização feita no df original

Iremos representar as redes pelas suas cores características, para facilitar o processo, trocamos cada nome no dataframe pela cor que queremos aplicar:

In [6]:
#atribuir cor de acordo com as redes
cores = {1:"blue",2:"red"} 

df.replace({'rede': cores},inplace=True)

Algumas categorias não eram úteis para o nosso problema, como "Molhos" e "Bebidas", pois queremos comparar os lanches dessas duas redes de fastfood e nesses casos encontramos produtos de terceiros. Além disso, renomeamos as categorias.

In [7]:
#df.categoria.unique()
cat_indesejadas = ["SAUCE","BREAKFAST","BEVERAGES","COFFEE & TEA"]
df = df[~df.categoria.isin(cat_indesejadas)]

#teste.categoria.unique()
categorias = ['WHOPPER', 'MEAT SANDWICH', 'CHICKEN SANDWICH', 'VEGGIES',
'SIDE DISH', 'ICE CREAM', 'SHAKES', 'BEEF & PORK',
       'CHICKEN & FISH', 'SALADS', 'SNACKS & SIDES', 'DESSERTS',
       'SMOOTHIES & SHAKES']

novas_categorias = ["SANDUICHE DE CARNE","SANDUICHE DE CARNE","SANDUICHE DE FRANGO/PEIXE","VEGGIES",
                    "ACOMPANHAMENTO","SOBREMESA","SHAKE","SANDUICHE DE CARNE",
                    "SANDUICHE DE FRANGO/PEIXE","ACOMPANHAMENTO","ACOMPANHAMENTO","SOBREMESA",
                   "SHAKE"]


df.replace(dict(zip(categorias,novas_categorias)),inplace=True)

Os NA podem ser trocados por 0 no nosso problema, pois de qualquer modo não serão representados pelos nodes. Caso fossem ignorados todos os dados que possuem NA, perderíamos muitas informações, pois muitos lanches seriam excluidos da base.

In [8]:
df.fillna(0,inplace=True)

Por fim, simplismente renomeamos as colunas dos nutrientes, pois serão os nomes utilizados nos nodes e no banco estão em inglês, além de alguns contem "_", o que não seria agradável na visualização do grafo.

In [9]:
# 'col' foi definido acima e seleciona somente as colunas dos nutrientes
col_names = ['calorias','gorduras totais','gorduras saturadas','gorduras trans','colesterol','sódio','carboidratos','fibras','açúcares','proteínas','vitamina A','vitamina C','cálcio','ferro']

df = df.rename(columns=dict(zip(col,col_names))) #renomeia as colunas dos nutrientes com os novos nomes (na lista col_names)

# Resultado

#### Após esses passos iniciais obtemos o seguinte data frame:

In [10]:
df

,calorias,gorduras totais,gorduras saturadas,gorduras trans,colesterol,sódio,carboidratos,fibras,açúcares,proteínas,vitamina A,vitamina C,cálcio,ferro,categoria,rede
name,,,,,,,,,,,,,,,,
WHOPPER,3.056074,2.556255,2.485836,1.000000,0.000000,1.001862,1.144379,2.175682,0.000000,3.042743,0.000000,0.0,0.000000,0.000000,SANDUICHE DE CARNE,blue
WHOPPER DUPLO,3.964147,3.578937,3.111451,1.000000,0.000000,1.001862,1.133684,2.763523,0.000000,4.478184,0.000000,0.0,0.000000,0.000000,SANDUICHE DE CARNE,blue
WHOPPER FURIOSO,3.545036,2.867506,2.407634,1.000000,0.000000,1.003725,1.157747,2.175682,0.000000,3.153162,0.000000,0.0,0.000000,0.000000,SANDUICHE DE CARNE,blue
WHOPPER JR.,2.120666,1.800360,1.078202,1.000000,0.000000,2.175114,1.085558,2.175682,0.000000,2.159395,0.000000,0.0,0.000000,0.000000,SANDUICHE DE CARNE,blue
WHOPPER JR. BARBECUE BACON,2.357555,1.978217,1.782019,1.000000,0.000000,2.614618,1.098926,2.175682,0.000000,2.490650,0.000000,0.0,0.000000,0.000000,SANDUICHE DE CARNE,blue
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MCFLURRY WITH OREO COOKIES (SMALL),2.548888,1.755895,1.703817,2.728201,1.514662,1.521445,1.213894,1.587841,3.229867,1.662511,1.614523,1.0,3.345656,1.916549,SHAKE,red
MCFLURRY WITH OREO COOKIES (MEDIUM),3.095555,2.022682,1.938423,1.345640,1.629031,1.707676,1.283410,1.587841,3.961542,1.828139,1.819365,1.0,3.932070,2.145686,SHAKE,red
MCFLURRY WITH OREO COOKIES (SNACK),2.032592,1.489109,1.469211,1.000000,1.343108,1.353838,1.141705,1.587841,2.498192,1.441674,1.409682,1.0,2.466035,1.687412,SHAKE,red


# Exportação do data frame

In [11]:
#df.to_csv("fastfood.csv")